### First Clusters

Neste arquivo foram gerados os primeiros clusters de autores utilizando dados da DBLP e um grafo simples de co-autorias. 1) O grafo é construído, 2) é gerada a entrada para o MCL e 3) o parsing de nomes dos autores a partir da saída do MCL.

Created in 2015-02-26 by Alberto Ueda

In [1]:
%pylab inline
%matplotlib inline

import pandas as pd
import numpy as np
import mmap
import time
import re

from unidecode import unidecode
UTF8 = 'utf-8'        

class Dict(dict):
    def __missing__(self, key):
            return False

#reload(sys)  # Solve the acentos problem, but create the terminal output problem
#sys.setdefaultencoding(UTF8)

mypath = '/mnt/hd0/alberto/large-datasets/dblp/dblp/coauthors-first-clusters/'

Populating the interactive namespace from numpy and matplotlib


In [5]:
a_p = pd.read_csv(mypath + "../authorpaper.csv", names=['author','paper'])#, nrows=17220)
a_p.head()

,author,paper
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [4]:
# this block takes a long time to run (5min)
groups = a_p.groupby('paper')['author'].apply(list) 
groups

paper
0                              [0, 189099]
1                 [0, 2029, 44493, 246705]
2                       [0, 22572, 111477]
3                                  [0, 21]
4          [0, 4590, 10861, 20004, 258270]
5                  [0, 6231, 12342, 21505]
6                   [0, 2029, 3133, 47070]
7                   [0, 4196, 6927, 25618]
8                   [0, 1530, 3811, 55613]
9                    [0, 1642, 2997, 6510]
10                      [0, 22945, 173529]
11                         [0, 178, 11697]
12                [0, 3937, 14286, 262524]
13       [0, 94, 686, 25318, 53553, 98074]
14                                 [0, 21]
...
4551347    [1595010]
4551348    [1595011]
4551349    [1595012]
4551350    [1595013]
4551351    [1595014]
4551352    [1595015]
4551353    [1595016]
4551354    [1595017]
4551355    [1595018]
4551356    [1595019]
4551357    [1595020]
4551358    [1595021]
4551359    [1595022]
4551360    [1595023]
4551361    [1595024]
Name: author, Length: 4551362,

In [3]:
groups.values

array([[0, 189099], [0, 2029, 44493, 246705], [0, 22572, 111477], ...,
       [1595022], [1595023], [1595024]], dtype=object)

###### To generate MCL input

In [96]:
def connect_coauthors(coauthors, f):
    for author1 in coauthors: 
        for author2 in coauthors:
            if (author1 < author2): 
                f.write(str(author1) + " " + str(author2) + '\n')
                #print author1, "--", author2            
               
with open(mypath + 'coauthors.csv', 'w') as f:                  
    k = 0
    for paper_coauthors in groups.values:
        #if k > 10: break  #max:4551362 # this takes a long time to run
        k += 1
        connect_coauthors(paper_coauthors, f)    

###### IDs ---> Names

In [3]:
# Encoding workaround
import sys
sys_out = sys.stdout
sys_err = sys.stderr
reload(sys)
sys.setdefaultencoding(UTF8)
sys.stdout = sys_out
sys.stderr = sys_err

a   = pd.read_csv(mypath + "../authors.csv", header=None, names=['a_name'], encoding=UTF8)#, nrows=100)
a['author'] = a.index
a_names = pd.merge(a_p, a, how='left')
a_names.head()


   author  paper           a_name
0       0      0  H. Vincent Poor
1       0      1  H. Vincent Poor
2       0      2  H. Vincent Poor
3       0      3  H. Vincent Poor
4       0      4  H. Vincent Poor


In [15]:
all_names = a_names[['author','a_name']].drop_duplicates().reset_index()
all_names.head()

,index,author,a_name
0,0,0,H. Vincent Poor
1,1095,1,Wei Wang
2,2136,2,Yan Zhang
3,3119,3,Wei Liu
4,4076,4,Wen Gao


###### Writing all_names.csv

In [34]:
all_names[['author','a_name']].to_csv(mypath + 'all_names.csv', index=False)

In [38]:
# All the indexes are also the author identifier
all_names[all_names['author'] != all_names.index]

,author,a_name


In [2]:
all_names = pd.read_csv(mypath + "all_names.csv")
author_id = 0
print all_names.ix[author_id]['a_name']

H. Vincent Poor


In [3]:
p_sigir = pd.read_csv(mypath + "papers.csv.sigir.new", header=None, names=['paper','conf', 'year', 'x'], sep='\t')#, nrows=17220)
p_sigir.head()

,paper,conf,year,x
0,1304,conf/sigir/GengLWWS11,2011,247
1,2415,journals/sigir/Zhang10,2010,624
2,2665,conf/sigir/YanWOKLZ11,2011,247
3,3705,conf/sigir/XuBCCHLL11,2011,247
4,3724,conf/sigir/MaLWSBWW14,2014,247


In [6]:
a_sigir = pd.merge(p_sigir, a_p, how='left', on='paper').merge(all_names, on='author', how='left')
a_sigir.head()

,paper,conf,year,x,author,a_name
0,1304,conf/sigir/GengLWWS11,2011,247,1,Wei Wang
1,1304,conf/sigir/GengLWWS11,2011,247,1190,Xiaodong Li
2,1304,conf/sigir/GengLWWS11,2011,247,64130,Guanggang Geng
3,1304,conf/sigir/GengLWWS11,2011,247,187812,Li-Ming Wang
4,1304,conf/sigir/GengLWWS11,2011,247,211944,Shuo Shen


In [7]:
a_sigir[a_sigir['a_name'].str.contains('Berthier A. Ribeiro-Neto')]

,paper,conf,year,x,author,a_name
5035,336214,conf/sigir/PossasZMR02,2002,247,10899,Berthier A. Ribeiro-Neto
5036,336317,conf/sigir/SaraivaMFMRZ01,2001,247,10899,Berthier A. Ribeiro-Neto
5037,365187,conf/sigir/LacerdaCGFZR06,2006,247,10899,Berthier A. Ribeiro-Neto
5038,638767,conf/sigir/RibeiroM96,1996,247,10899,Berthier A. Ribeiro-Neto
5039,900444,conf/sigir/FernandesMSRA11,2011,247,10899,Berthier A. Ribeiro-Neto
5040,953989,conf/sigir/BadueBGRZ05,2005,247,10899,Berthier A. Ribeiro-Neto
5041,953992,conf/sigir/Ribeiro-NetoMNZ99,1999,247,10899,Berthier A. Ribeiro-Neto
5042,954000,conf/sigir/SilvaRCMZ00,2000,247,10899,Berthier A. Ribeiro-Neto
5043,1127013,conf/sigir/Ribeiro-NetoCGM05,2005,247,10899,Berthier A. Ribeiro-Neto


In [9]:
a_sigir_unique = a_sigir[['author','a_name']].drop_duplicates().reset_index()
a_sigir_unique['score'] = 1 # len(a_sigir[a_sigir['author'] == a_sigir_unique['author']])
a_sigir_unique[['author','a_name', 'score']].to_csv(mypath + 'a_sigir_unique.csv', index=False)
a_sigir_unique.head()

,index,author,a_name,score
0,0,1,Wei Wang,1
1,1,1190,Xiaodong Li,1
2,2,64130,Guanggang Geng,1
3,3,187812,Li-Ming Wang,1
4,4,211944,Shuo Shen,1


In [11]:
print len(a_sigir), len(a_sigir_unique)

11561 5040


In [14]:
list_a_sigir = a_sigir_unique['author'].tolist()
-10899 in list_a_sigir

False

###### AFTER MCL Operations

In [15]:
# example: time mcl coauthors.csv.unique.10k --abc -o out.mcl -te 4 -scheme 6
with open(mypath + 'out.mcl.all', 'r') as f, open(mypath + 'clusters-authors-all-sigir-mark.csv', 'w') as clusters:       
    k = 1
    for line in f:
        clusters.write("Cluster #" + str(k) + ",")
        for author_id in line.split():
            # if (k < 2): print author_id, all_names.ix[int(author_id)]['a_name'] #Cluster #1
            
            sigir_mark = ''             
            if int(author_id) in list_a_sigir: 
                sigir_mark = '*'            
            
            clusters.write("[" + all_names.ix[int(author_id)]['a_name'] 
                           + sigir_mark
                           + "],")
        clusters.write('\n')
        k += 1
 
# To count the '*'s per line:
# grep '*' clusters-authors-all-sigir-mark.csv -on | cut -d : -f 1 | uniq -c | sort -rn | head

# Top clusters
# grep '*' clusters-authors-all-sigir-mark.csv -on | cut -d : -f 1 | uniq -c | sort -rn | awk '{print $2}' | head -n 20 > sigir-top-clusters-only.txt 

# Print top cluster lines 
# sed -n '705p;4137p;157p;659p;3580p;491p;863p;680p;4361p;724p;2650p' clusters-authors-all-sigir-mark.csv > top-sigir-clusters.txt


 